# Home Credit Scorecard Model 
  

## Introduction

Notebook untuk pemodelan skor kredit Home Credit Indonesia dalam Data Scientist Project-Based Internship with Rakamin.

Diberikan 7 dataset yang akan digunakan dalam pemodelan ini:
1. __application__
    
   Dataset utama yang berisi informasi pengajuan kredit.
   - 1 - Nasabah yang mengalami kesulitan pembayaran: misal mengalami keterlambatan pembayaran lebih dari X hari untuk setidaknya satu dari Y angsuran (installment) pertama pinjaman dalam sampel
   - 0 - semua kasus lainnya
2. __bureau__
    
   Dataset yang berisi informasi kredit yang diambil oleh pelanggan di lembaga keuangan lain.
3. __bureau_balance__

    Dataset yang berisi informasi pembayaran kredit di lembaga keuangan lain. 
4. __credit_card_balance__

    Dataset yang berisi saldo bulanan dari kartu kredit yang dimiliki oleh pelanggan dengan Home Credit.
5. __installments_payments__

    Dataset yang berisi riwayat pembayaran cicilan kredit yang diberikan oleh Home Credit. 
6. __pos_cash_balance__

    Dataset yang berisi saldo bulanan dari kredit POS dan kredit tunai yang dimiliki oleh pelanggan dengan Home Credit.
7. __previous_application__

    Dataset yang berisi riwayat pengajuan kredit oleh pelanggan dengan Home Credit.

## Objective

Tujuan dari notebook ini adalah membangun model skor kredit menggunakan data yang diberikan. Model ini akan digunakan untuk memprediksi apakah pengajuan kredit akan disetujui atau tidak.

Data utama yang digunakan adalah application_train.csv. 


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

import pandas as pd
import sqlite3
import seaborn as sns
import matplotlib.pyplot as plt

plt.style.use('ggplot')

In [ ]:
conn = sqlite3.connect('hci_application.db')

# Exploratory Data Analysis

## Initial EDA

In [ ]:
def fetch_random_n_rows(n, table_name, primary_key) -> pd.DataFrame:
    return pd.read_sql(f"""SELECT *
                       FROM {table_name}
                       WHERE {primary_key}
                       IN (
                        SELECT {primary_key}
                        FROM {table_name}
                        ORDER BY RANDOM()
                        LIMIT {n}
                       )""", conn)


In [ ]:
df = pd.read_sql('SELECT * FROM application', conn)
print(df.shape)
df.head()

In [ ]:
df.info(verbose=True)

In [ ]:
df.describe()

In [ ]:
# get a random sample of table for only 100 rows
prev_df = fetch_random_n_rows(100, 'previous_application', 'SK_ID_PREV')
print(prev_df.shape)
prev_df.head()

In [ ]:
prev_df.info(verbose=True)

In [ ]:
bureau_df = fetch_random_n_rows(100, 'bureau', 'SK_ID_BUREAU')
print(bureau_df.shape)
bureau_df.head()

In [ ]:
bureau_df.info(verbose=True)

In [ ]:
bureau_df.describe()

## Data Cleaning

### Application Dataset

Dataset utama untuk setiap aplikasi pinjaman yang disetujui atau ditolak ditunjukkan dengan kolom TARGET.

- 1 - Nasabah yang mengalami kesulitan pembayaran: misal mengalami keterlambatan pembayaran lebih dari X hari untuk setidaknya satu dari Y angsuran (installment) pertama pinjaman dalam sampel
- 0 - semua kasus lainnya

In [ ]:
missing_data = df.isnull()
missing_data.head(5)

In [ ]:
for column in missing_data.columns.values.tolist():
    if missing_data[column].value_counts().get(True):
        print(missing_data[column].value_counts(), '\n')

In [ ]:
excluded_column = ['SK_ID_CURR', 'TARGET', 'AMT_ANNUITY', 'COMMONAREA_AVG']

#### Drop Column

Too much missing value

- OWN_CAR_AGE
- EXT_SOURCE_1
- APARTMENTS_AVG
- BASEMENTAREA_AVG
- YEARS_BEGINEXPLUATATION_AVG
- YEARS_BUILD_AVG
- ELEVATORS_AVG
- ENTRANCES_AVG
- FLOORSMAX_AVG
- LIVINGAREA_AVG
- 

In [ ]:
def has_too_many_missing_values(column: pd.Series, threshold=0.45):
    # A column is considered to have too many missing values if more than a certain threshold (e.g., 40%, almost half of it) of its values are missing
    return column.isnull().sum() / df.shape[0] > threshold


# drop row if it has too many missing values
df = df.drop(columns=[column for column in df.columns if
                      has_too_many_missing_values(df[column]) and column not in excluded_column])
df.head()

#### Custom Impute

- AMT_ANNUITY
- COMMONAREA_AVG -> fill na to 0

Untuk AMT_ANNUITY dapat diimpute dengan nilai rata-rata dari AMT_ANNUITY pada dataset previous_application.

In [ ]:
df[df['AMT_ANNUITY'].isnull()]

In [ ]:
prev_df_amt_annuity = pd.read_sql('SELECT SK_ID_CURR, AMT_ANNUITY FROM previous_application', conn)
prev_df_amt_annuity.head()

In [ ]:
prev_df_amt_annuity_null = prev_df_amt_annuity.merge(df[df['AMT_ANNUITY'].isnull()], how='right', on='SK_ID_CURR')
prev_df_amt_annuity_null.head()

In [ ]:
mean_amt_annuity = prev_df_amt_annuity_null[['SK_ID_CURR', 'AMT_ANNUITY_x']].groupby('SK_ID_CURR').mean()
for key, value in mean_amt_annuity.AMT_ANNUITY_x.to_dict().items():
    df.loc[df['SK_ID_CURR'] == key, 'AMT_ANNUITY'] = value

In [ ]:
df.AMT_ANNUITY.isnull().sum()

Untuk COMMONAREA_AVG dapat diimpute dengan nilai 0 karena merupakan rata-rata luas area bersama dari apartemen klien dan nilai nya sangat skewed ke 0.

In [ ]:
df.COMMONAREA_AVG.isnull().sum()

In [ ]:
df.COMMONAREA_AVG.plot(kind='hist', bins=50, figsize=(10, 5))

In [ ]:
df.COMMONAREA_AVG.fillna(0, inplace=True)

#### Median Impute

Jika tipe data numerik dan kontinu. Median dipilih karena kebanyakan kolom memiliki distribusi yang skewed. Contoh:

- AMT_GOODS_PRICE
- EXT_SOURCE_2
- EXT_SOURCE_2
- FLOORSMIN_AVG
- LANDAREA_AVG
- LIVINGAPARTMENTS_AVG
- ...


In [ ]:
df['AMT_GOODS_PRICE'].plot(kind='hist', bins=50, figsize=(10, 5))

In [ ]:
df[['OWN_CAR_AGE', 'TARGET']].corr()

In [ ]:
df[['EXT_SOURCE_2', 'TARGET']].corr()

In [ ]:
df[['EXT_SOURCE_3', 'TARGET']].corr()

In [ ]:
df[['FLOORSMIN_AVG', 'TARGET']].corr()

In [ ]:
df['FLOORSMIN_AVG'].plot(kind='hist', bins=50, figsize=(10, 5))

In [ ]:
df.LANDAREA_AVG.plot(kind='hist', bins=50, figsize=(10, 5))

In [ ]:
def fill_na_with_median(column: pd.Series):
    return column.fillna(column.median())


def is_numeric_and_continuous(column: pd.Series):
    return column.dtype.kind in 'fi' and column.nunique() / column.shape[0] > 0.01


df = df.apply(lambda x: fill_na_with_median(x) if is_numeric_and_continuous(x) and x.name not in excluded_column else x)
df.head()

#### Mode Impute

Untuk kolom kategorikal yang bertipe objek atau numerik. Contoh:

- NAME_TYPE_SUITE
- OCCUPATION_TYPE
- CNT_FAM_MEMBERS
- NONLIVINGAPARTMENTS_AVG
- ...

In [ ]:
df['NAME_TYPE_SUITE'].value_counts()

In [ ]:
df['OCCUPATION_TYPE'].value_counts().plot(kind='bar', figsize=(10, 5))

In [ ]:
df['NONLIVINGAPARTMENTS_AVG'].plot(kind='hist', bins=50, figsize=(10, 5))

In [ ]:
def is_categorical(column: pd.Series):
    return column.dtype.kind in 'O' and column.nunique() / column.shape[0] < 0.01


def fill_na_with_mode(column: pd.Series):
    return column.fillna(column.mode()[0])


df = df.apply(lambda x: fill_na_with_mode(x) if is_categorical(x) and x.name not in excluded_column else x)
df.head()

#### Leftover Missing Values

This means that the column has missing values that are not too many, and the column is not numeric and continuous. For these columns, we can impute the missing values with the median for numeric columns and the mode for categorical columns.

In [ ]:
df.isnull().sum()

In [ ]:
df.AMT_GOODS_PRICE.plot(kind='hist', bins=50, figsize=(10, 5))

In [ ]:
df.AMT_GOODS_PRICE.fillna(df.AMT_GOODS_PRICE.median(), inplace=True)

In [ ]:
df.CNT_FAM_MEMBERS.plot(kind='hist', bins=50, figsize=(10, 5))

In [ ]:
df.CNT_FAM_MEMBERS.fillna(df.CNT_FAM_MEMBERS.median(), inplace=True)

In [ ]:
df.EXT_SOURCE_3.plot(kind='hist', bins=50, figsize=(10, 5))

In [ ]:
df.EXT_SOURCE_3.fillna(df.EXT_SOURCE_3.median(), inplace=True)

In [ ]:
df.OBS_30_CNT_SOCIAL_CIRCLE.plot(kind='hist', bins=50, figsize=(10, 5))

In [ ]:
df.OBS_30_CNT_SOCIAL_CIRCLE.fillna(0, inplace=True)

In [ ]:
df.DEF_30_CNT_SOCIAL_CIRCLE.plot(kind='hist', bins=50, figsize=(10, 5))

In [ ]:
df.fillna({'DEF_30_CNT_SOCIAL_CIRCLE': 0, 'DEF_60_CNT_SOCIAL_CIRCLE': 0, 'OBS_60_CNT_SOCIAL_CIRCLE': 0}, inplace=True)

In [ ]:
req_bureau_col = [x for x in df.columns if x.startswith('AMT_REQ')]

# visualize its distribution
fig, axes = plt.subplots(3, 2, figsize=(15, 15))
axes = axes.flatten()

for i, col in enumerate(req_bureau_col):
    df[col].plot(kind='hist', bins=50, ax=axes[i])
    axes[i].set_title(col)
    
plt.tight_layout()
plt.show()

In [ ]:
# most of the columns can be filled with 0, except for AMT_REQ_CREDIT_BUREAU_YEAR.
# AMT_REQ_CREDIT_BUREAU_YEAR will be filled with median
df.fillna({col: 0 for col in req_bureau_col[:-1]}, inplace=True)
df.fillna({'AMT_REQ_CREDIT_BUREAU_YEAR': df['AMT_REQ_CREDIT_BUREAU_YEAR'].median()}, inplace=True)

In [ ]:
df.isnull().sum()

### Bureau Dataset

For bureau dataset, we can immediately join it with bureau_balance dataset to get the latest status of each credit bureau.
 
We will perform average, maximum, and count aggregation for MONTHS_BALANCE, and count aggregation for each STATUS code.

But first lets check if the identifier in bureau_balance is also available in the bureau dataset.

In [ ]:
bureau_balance_id = pd.read_sql('SELECT DISTINCT SK_ID_BUREAU FROM bureau_balance', conn)
bureau_balance_id.head()

In [ ]:
# check if the SK_ID_BUREAU in bureau_balance is also available in bureau
bureau_id = pd.read_sql('SELECT DISTINCT SK_ID_BUREAU FROM bureau', conn)
bureau_id.head()

In [112]:
print('Number of unique SK_ID_BUREAU in bureau_balance:', bureau_balance_id.shape[0])
print('Number of unique SK_ID_BUREAU in bureau:', bureau_id.shape[0])
print('Number of unique SK_ID_BUREAU in bureau_balance that is also available in bureau:', bureau_balance_id.SK_ID_BUREAU.isin(bureau_id.SK_ID_BUREAU).sum())
print('Percentage of SK_ID_BUREAU in bureau_balance that is also available in bureau:', bureau_balance_id.SK_ID_BUREAU.isin(bureau_id.SK_ID_BUREAU).sum() / bureau_id.shape[0])

Number of unique SK_ID_BUREAU in bureau_balance: 817395
Number of unique SK_ID_BUREAU in bureau: 1716428
Number of unique SK_ID_BUREAU in bureau_balance that is also available in bureau: 774354
Percentage of SK_ID_BUREAU in bureau_balance that is also available in bureau: 0.4511427219784343


In [ ]:
cur = conn.cursor()

In [ ]:
agg_query = """
    CREATE TEMPORARY VIEW bureau_balance_agg AS
    SELECT
        SK_ID_BUREAU,
        AVG(MONTHS_BALANCE) AS MONTHS_BALANCE_mean,
        MAX(MONTHS_BALANCE) AS MONTHS_BALANCE_max,
        COUNT(MONTHS_BALANCE) AS MONTHS_BALANCE_count,
        SUM(CASE WHEN STATUS = '0' THEN 1 ELSE 0 END) AS STATUS_0_count,
        SUM(CASE WHEN STATUS = '1' THEN 1 ELSE 0 END) AS STATUS_1_count,
        SUM(CASE WHEN STATUS = '2' THEN 1 ELSE 0 END) AS STATUS_2_count,
        SUM(CASE WHEN STATUS = '3' THEN 1 ELSE 0 END) AS STATUS_3_count,
        SUM(CASE WHEN STATUS = '4' THEN 1 ELSE 0 END) AS STATUS_4_count,
        SUM(CASE WHEN STATUS = '5' THEN 1 ELSE 0 END) AS STATUS_5_count,
        SUM(CASE WHEN STATUS = 'C' THEN 1 ELSE 0 END) AS STATUS_C_count,
        SUM(CASE WHEN STATUS = 'X' THEN 1 ELSE 0 END) AS STATUS_X_count
    FROM
        bureau_balance
    GROUP BY
        SK_ID_BUREAU;
"""

cur.execute(agg_query)

In [ ]:
join_query = """
    SELECT
        b.*,
        bb.MONTHS_BALANCE_mean,
        bb.MONTHS_BALANCE_max,
        bb.MONTHS_BALANCE_count,
        bb.STATUS_0_count,
        bb.STATUS_1_count,
        bb.STATUS_2_count,
        bb.STATUS_3_count,
        bb.STATUS_4_count,
        bb.STATUS_5_count,
        bb.STATUS_C_count,
        bb.STATUS_X_count
    FROM
        bureau b
    LEFT JOIN
        bureau_balance_agg bb
    ON
        b.SK_ID_BUREAU = bb.SK_ID_BUREAU
    LIMIT
        100;
"""

bureau_df = pd.read_sql(join_query, conn)
bureau_df.head()

In [120]:
bureau_df.isna().sum()

SK_ID_CURR                      0
SK_ID_BUREAU                    0
CREDIT_ACTIVE                   0
CREDIT_CURRENCY                 0
DAYS_CREDIT                     0
CREDIT_DAY_OVERDUE              0
DAYS_CREDIT_ENDDATE        105553
DAYS_ENDDATE_FACT          633653
AMT_CREDIT_MAX_OVERDUE    1124488
CNT_CREDIT_PROLONG              0
AMT_CREDIT_SUM                 13
AMT_CREDIT_SUM_DEBT        257669
AMT_CREDIT_SUM_LIMIT       591780
AMT_CREDIT_SUM_OVERDUE          0
CREDIT_TYPE                     0
DAYS_CREDIT_UPDATE              0
AMT_ANNUITY               1226791
MONTHS_BALANCE_mean        942074
MONTHS_BALANCE_max         942074
MONTHS_BALANCE_count       942074
STATUS_0_count             942074
STATUS_1_count             942074
STATUS_2_count             942074
STATUS_3_count             942074
STATUS_4_count             942074
STATUS_5_count             942074
STATUS_C_count             942074
STATUS_X_count             942074
dtype: int64

In [115]:
bureau_df

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,...,MONTHS_BALANCE_max,MONTHS_BALANCE_count,STATUS_0_count,STATUS_1_count,STATUS_2_count,STATUS_3_count,STATUS_4_count,STATUS_5_count,STATUS_C_count,STATUS_X_count
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716423,259355,5057750,Active,currency 1,-44,0,-30.0,NaN,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1716424,100044,5057754,Closed,currency 1,-2648,0,-2433.0,-2493.0,5476.5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1716425,100044,5057762,Closed,currency 1,-1809,0,-1628.0,-970.0,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1716426,246829,5057770,Closed,currency 1,-1878,0,-1513.0,-1513.0,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [121]:
drop_view_query = "DROP VIEW IF EXISTS bureau_balance_agg;"
cur.execute(drop_view_query)

#### Drop Column

In [ ]:
excluded_column = ['SK_ID_CURR', 'SK_ID_BUREAU']